In [1]:
import pandas as pd
from os import getcwd, listdir, makedirs
from os.path import join, isfile, exists, abspath, pardir
from sys import path
# from pandas_profiling import ProfileReport

##### Configs

In [2]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, "data")
scripts_dir = join(parent_dir, "src", "scripts")
data_file = join(data_dir, "publications.csv")

In [3]:
path.append(scripts_dir)
from connect import Neo4jConnection

##### Load data

In [4]:
df = pd.read_csv(data_file)
df.head(3)

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,Document Type,Publication Stage,Access Type,Source,EID
0,"Gautam A., Crandall J.W., Goodrich M.A.",57218202833;7004904337;7005513246;,Self-assessment of Proficiency of Intelligent ...,2021,Advances in Intelligent Systems and Computing,1210 AISC,NaN,NaN,108,113,...,"Computer Science Department, Brigham Young Uni...","Gautam, A., Computer Science Department, Brigh...","Autonomous systems, although capable of perfor...",Environment; Goal(s); Intelligent agents; Prof...,Drones; Human engineering; Intelligent systems...,Conference Paper,Final,NaN,Scopus,2-s2.0-85088238482
1,"de Moura Oliveira P.B., Hedengren J.D., Boaven...",6508306234;9277159100;6507358470;,Bridging theory to practice: Feedforward and c...,2021,Lecture Notes in Electrical Engineering,695 LNEE,NaN,NaN,23,32,...,"INESC-TEC Technology and Science, Campus da FE...","de Moura Oliveira, P.B., INESC-TEC Technology ...",Practice is of the essence in Engineering cour...,NaN,Automation; Cascade control systems; Computati...,Conference Paper,Final,NaN,Scopus,2-s2.0-85091306533
2,"Hajimirzaie S.M., Hotchkiss R.H.",53879700900;26642910500;,Development of sediment management guidelines ...,2020,Journal of Hydraulic Engineering,146,12,02520004,NaN,NaN,...,"Task Committee Secretary and Lead Engineer, Op...","Hajimirzaie, S.M., Task Committee Secretary an...",Forum papers are thought-provoking opinion pie...,NaN,Hydraulics; Editorial board; Sediment manageme...,Review,Final,NaN,Scopus,2-s2.0-85091917202


In [5]:
# profile = ProfileReport(df, title="Publications")
# profile.to_widgets()

In [6]:
database = "sdm"
driver = Neo4jConnection(uri="bolt://localhost:7687", user=None, pwd=None, database=database)

In [19]:
result = driver.query("""
Match (n) return n
""")

In [15]:
r = result.single()

In [20]:
list(result)

[]